# Shorts Maker

#### From two raw videos:
1. Call ShortsFusion to merge them,
2. Call Transcriber to create subtitles,
3. Call SubtitleStyler to apply subtitles.

In [1]:
# Environment Setup
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../..')  # Add project root to path

## Imports

In [2]:
from content_creation_tools.utils.shorts_fusion import merge_videos
from content_creation_tools.models.transcriber import FasterWhisper
from content_creation_tools.models.subtitle_styler import SubtitleStyler

/home/ckaradeniz/miniconda3/envs/VU/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## ShortsFusion

Merge two videos

In [3]:
video1_path = '../../examples/shorts_test.mp4'
video2_path = '../../examples/shorts_test.mp4'

output_path = '../../examples/shorts_fusion.mp4'

In [ ]:

merge_videos(
    video1_path=video1_path,
    video2_path=video2_path,
    t1_start=0, t1_end=5,
    t2_start=0, t2_end=5,
    output_path=output_path,
    line_color=(255, 0, 0),
    line_width=5,
    speed_factor=1.2,
    target_resolution=(1920, 1080)
)


{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'mp42', 'minor_version': '0', 'compatible_brands': 'isommp42', 'creation_time': '2025-01-16T16:45:33.000000Z', 'encoder': 'Google'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [360, 640], 'bitrate': 328, 'fps': 30.0, 'codec_name': 'h264', 'profile': '(Main)', 'metadata': {'Metadata': '', 'creation_time': '2025-01-16T16:45:33.000000Z', 'handler_name': 'ISO Media file produced by Google Inc. Created on: 01/16/2025.', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 44100, 'bitrate': 48, 'metadata': {'Metadata': '', 'creation_time': '2025-01-16T16:45:33.000000Z', 'handler_name': 'ISO Media file produced by Google Inc. Created on: 01/16/2025.', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 16.16, 'bitrate': 377, 'start': 0.0, 'd

MoviePy - Done.
MoviePy - Writing video ../../examples/shorts_fusion.mp4



MoviePy - Done !
MoviePy - video ready ../../examples/shorts_fusion.mp4


## Transcriber

#### Create subtitles file using FasterWhisper

Set Parameters:

In [4]:
faster_whisper = FasterWhisper()

print(faster_whisper.get_parameter_help())
set_params = {'log_progress': True}

params = faster_whisper.validate_parameters(set_params)
print(params)


FasterWhisper Parameters:
Parameter            Type            Description                    Default   
model_size_or_path   str             Size of the model to use, a path to a converted model directory, or a CTranslate2-converted Whisper model ID from the HF Hub. medium    
device               str             Device to use for computation ("cpu", "cuda", "auto"). auto      
language             str             The language spoken in the audio such as "en" or "fr". If not set, the language will be detected in the first 30 seconds of audio. tr        
log_progress         bool            Whether to log progress to the console. False     
max_initial_timestamp float           The initial timestamp cannot be later than this. 10        
max_chars_per_line   int             Maximum number of characters per line in the subtitles. 20        

{'log_progress': True, 'model_size_or_path': 'medium', 'device': 'auto', 'language': 'tr', 'max_initial_timestamp': 10, 'max_chars_per_line': 20}


In [5]:
from content_creation_tools.utils import extract_audio, write_srt_file

In [6]:
output_audio_path = extract_audio(output_path)


{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf61.1.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [270, 1920], 'bitrate': 594, 'fps': 25.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]', 'encoder': 'Lavc61.3.100 libx264'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 44100, 'bitrate': 128, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 4.2, 'bitrate': 733, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [270, 1920], 'video_bitrate': 594, 'video_fps': 25.0, 'defa

MoviePy - Done.
Successfully extracted audio to: ../../examples/shorts_fusion_audio.mp3


In [ ]:
merged_segments = faster_whisper.generate(params, output_audio_path)
# write_srt_file(merged_segments, output_audio_path.replace('.mp3', '.srt'))


╭─ Initializing the model...
Parameters: {'log_progress': True, 'model_size_or_path': 'medium', 'device': 'auto', 'language': 'tr', 'max_initial_timestamp': 10, 'max_chars_per_line': 20}


[2025-03-28 10:23:19.186] [ctranslate2] [thread 18239] [warning] The compute type inferred from the saved model is float16, but the target device or backend do not support efficient float16 computation. The model weights have been automatically converted to use the float32 compute type instead.


╭─ Transcribing the audio...


100%|██████████| 4.2/4.2 [00:26<00:00,  6.32s/seconds]


## Subtitle Styler

#### Apply subtitles to the merged video using SubtitleStyler

In [8]:
sub_styler = SubtitleStyler()

print(sub_styler.get_parameter_help())
set_params = {'font_size': 30}

params = sub_styler.validate_parameters(set_params)
print(params)


SubtitleStyler Parameters:
Parameter            Type            Description                    Default   
font                 str             Font name (e.g., "Arial-Bold"). Arial_Bold
font_size            int             Font size.                     14        
color                str             Text color (hex or name).      white     
bg_color             tuple           Background color (RGB).        (0, 0, 0) 
bg_opacity           float           Background opacity (0.0 to 1.0). 0.6       
stroke_color         str             Text stroke/border color.      red       
stroke_width         int             Stroke width.                  1         
position             tuple           Text position (e.g., ("center", "bottom", "top")). ('center', 'bottom')
padding              tuple           Background padding (horizontal, vertical). (10, 5)   

{'font_size': 30, 'font': 'Arial_Bold', 'color': 'white', 'bg_color': (0, 0, 0), 'bg_opacity': 0.6, 'stroke_color': 'red', 'stroke_width

In [ ]:
final_video = sub_styler.generate(
    params=params,
    subtitles=merged_segments,
    video_path=output_path,
)

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf61.1.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [270, 1920], 'bitrate': 594, 'fps': 25.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]', 'encoder': 'Lavc61.3.100 libx264'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 44100, 'bitrate': 128, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 4.2, 'bitrate': 733, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [270, 1920], 'video_bitrate': 594, 'video_fps': 25.0, 'defa

In [10]:

final_video.write_videofile(output_path.replace('.mp4', '_result.mp4'), fps=final_video.fps, codec="libx264")

MoviePy - Building video ../../examples/shorts_fusion_result.mp4.
MoviePy - Writing audio in shorts_fusion_resultTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
MoviePy - Writing video ../../examples/shorts_fusion_result.mp4



MoviePy - Done !
MoviePy - video ready ../../examples/shorts_fusion_result.mp4
